# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias   
Número:  53746       Nome:   Andrei Tataru  
Número:  51127       Nome:   Luís Ferreirinha  

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação dos estados

(descreva aqui, textualmente, como decidiu representar os estados em Python; ilustre nas células de código abaixo a representação em Python de um estado à sua escolha)

Para representar-mos os estados em python, decidimos criar uma classe EspacoPacman, onde cada objeto desta classe representa um estado do nosso problema, para este problema em concreto decidimos que estes objetos devem guardar a posição do pacman, o número de pontos acumulados, o tempo desde a ultima captura de uma pastilha, casas visitadas e número de vezes que foram visitadas e finalmente as coordenadas e tipo das pastilhas que ainda se encontram no campo.

- Representação dos Estados:  
    - Posição Pacman: tuple $\left(x,y\right)$ onde $x,y$ são inteiros
    - Pontos: inteiro p
    - Tempo: inteiro t
    - Casas Visitadas: dicionário onde os elementos têm a seguinte forma $\{(x,y) : n\}$, $x,y,n$ são inteiros e $n$ representa o número de visitas a essa casa.
    - Conjunto das Pastilhas: dicionário onde os elementos têm a seguinte forma $\{(x,y) : tipo\}$, $x,y$ são inteiros e tipo é um character que representa o tipo da pastilha


In [36]:
# se definiu uma classe para representar os estados, inclua aqui o código Python correspondente
import copy

class EstadoPacman():
    """Objetos desta classe representam um estado do espaço do problema Pacman comilao
    """

    def __init__(self, posicao_pacman, pastilhas, pontos=0, tempo=0, visitas={}):
        self.pos = posicao_pacman
        self.pontos = pontos
        self.tempo = tempo
        self.pastilhas = copy.deepcopy(pastilhas)
        if visitas == {}:
            self.visitas = {posicao_pacman : 1}
        else:
            self.visitas = copy.deepcopy(visitas)

    def __lt__(self, other):
        return self.pontos < other.pontos
            
    def __eq__(self,other):
        if not isinstance(other, EstadoPacman): return False
        return other.pos == self.pos and other.pontos == self.pontos and other.tempo == self.tempo \
            and other.pastilhas == self.pastilhas and other.visitas == self.visitas
    
    def __hash__(self):
        return hash((self.pos, self.pontos, self.tempo)) + \
               hash(frozenset(self.pastilhas)) + \
               hash(frozenset(self.visitas))

    def __repr__(self):
        return str([self.pos, self.pontos, self.tempo, self.pastilhas, self.visitas])

    def __str__(self):
        return f"Posição Pacman: {self.pos}\nPontos: {self.pontos}\nTempo desde última pastilha: {self.tempo}\
                \nPastilhas restantes: {self.pastilhas}\nCasas Visitadas: {self.visitas}"


Vamos representar o seguinte estado:
-   Pacman está em (1,2)
-   Tem 2 pontos
-   Passaram 2 unidades de tempo desda a última pastilha comida
-   Restam as seguintes pastilhas: {(5,6) : "N", (3,2) : "D"}
-   Foram visitadas as seguintes casas: {(0,0) : 1, (1,0) : 1, (1,1) : 1, (1,2) : 1}

In [31]:
# representação de um estado à sua escolha
# Criamos um objecto do tipo EstadoPacman com os parametros anteriores
pastilhas_exemplo = {(5,6) : "N", (3,2) : "D"}
visitas_exemplo = {(0,0) : 1, (1,0) : 1, (1,1) : 1, (1,2) : 1}
estado_exemplo = EstadoPacman((1,2), pastilhas_exemplo , 2, 2, visitas_exemplo)

True

Como foi definido o método ```__str__``` para esta classe podemos imprimir o objecto para visualizar melhor o estado

In [40]:
print(estado_exemplo)

Posição Pacman: (1, 2)
Pontos: 2
Tempo desde última pastilha: 2                
Pastilhas restantes: {(5, 6): 'N', (3, 2): 'D'}
Casas Visitadas: {(0, 0): 1, (1, 0): 1, (1, 1): 1, (1, 2): 1}


## Formulação do problema

(explique textualmente como decidiu formular em Python o seu problema)

- Goal Test: Pontos Totais = Pontos Objetivo

- Operadores:  
    - Mover-se numa das quatro direcções ortogonais para as quatro células vizinhas, se não for impedido pelos limites do mundo nem por obstáculos:  
        - Norte: $(x,y) \to (x+1,y)$
        - Sul: $(x,y) \to (x-1,y$
        - Este: $(x,y) \to (x,y+1)$
        - Oeste: $(x,y) \to (x,y-1)$
    - Se a casa onde o pacman se encontra contem uma pastilha, ele come a pastilha, removendo esta e adicionando os respetivos pontos ao seu total de pontos:
        - Pontos Totais = Pontos Totais + Pontos da Pastilha

In [35]:
from searchPlus import *
import copy

class PacmanPastilhas(Problem):

    moves = {"N" : (0,-1), "S" : (0,1), "E" : (-1,0), "O" : (1,0)}

    def __init__(self, pos, objetivo, pastilhas, obstaculos, dimensao):
        
        #super().__init__(EstadoPacman(pos, pastilhas), goal=objetivo)
        self.initial = EstadoPacman(pos, pastilhas)
        self.goal = objetivo
        self.obstacles = obstaculos
        self.dim = dimensao
        

    def calculate_points(time, pastilha):
        if pastilha == "N":
            return 1
        elif pastilha == "D":
            return max(0,5-time)
        else:
            return time
    
    def calculate_position(pos, direction):
        return (pos[0]+self.moves[direction][0], pos[1]+self.moves[direction][1])

    def actions(self, state):
        return [d for d in self.moves.keys() if self.calculate_position(state.pos,d) not in self.obstacles]

    def result(self, state, action):
        new_position = self.calculate_position(state.pos, action)
        new_visitas = copy.deepcopy(state.visitas)
        if new_position in new_visitas.keys():
            new_visitas[new_position] += 1
        else:
            new_visitas[new_position] = 1

        if new_position in state.pastilhas.keys():
            new_points = state.pontos + self.calculate_points(state.tempo, state.pastilhas[new_position])
            new_pastilhas = copy.deepcopy(state.pastilhas)
            new_pastilhas.pop(new_position)
            new_time = 0
            return EstadoPacman(new_position, new_pastilhas, new_points, new_time, new_visitas)
        
        return EstadoPacman(new_position, state.pastilhas, state.pontos, state.tempo+1, new_visitas)


    def path_cost(self, cost, state, action, next_state):
        return cost+next_state.visitas[next_state.pos]

    def display(self, state):
        board = ""
        for y in range(self.dim):
            for x in range(self.dim):
                if state.pos == (x,y):
                    board += "@ "
                elif (x,y) in self.obstacles:
                    board += "# "
                elif (x,y) in state.pastilhas.keys():
                    board += f"{state.pastilhas[(x,y)]} "
                else:
                    board += ". "
            board += "\n"
        print(board)

    def goal_test(self, state):
        return state.points == self.goal

## Criação de estados e do problema

(Mostrem que o código está a funcionar, construindo instâncias da classe **PacmanPastilhas**, fazendo display dos estados, verificando o teste do estado final, gerando as ações para alguns estados, executando ações a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as ações (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de ações está a funcionar bem.)

In [ ]:
# código de teste do problema

def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

def exec(p,estado,accoes):
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
    p.display(estado)
    print('Custo:',custo)
    print('Goal?',p.goal_test(estado))
    return (estado,custo)

## Teste de procura de solução

(utilização de algoritmos de procura aprendidos nas aulas e comparação dos resultados ao nível de tempo de execução e solução obtida; comente aqui os resultados obtidos e o que observa)

In [ ]:
# código de aplicação dos algoritmos